# STEP-1: Test Steps clustering with Word2Vec

1. Text embedding technique: Word2Vec
2. Text similarity: Word Mover's Distance (WMD)
3. Clustering: K-means

In [1]:
# Import necessary libraries
import os
import gc

import pandas as pd
pd.options.mode.chained_assignment = None

import numpy as np
import math
import statistics as st
import re
import string
import time
import matplotlib.pyplot as plt

# For word frequency
from collections import defaultdict

# ML libraries
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
import scipy.cluster.hierarchy as sch

# Gensim
import gensim
from pyemd import emd
from gensim.similarities import WmdSimilarity
from gensim.models import Word2Vec, Phrases, KeyedVectors

# NLP libraries
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import RegexpTokenizer, word_tokenize, TweetTokenizer
from nltk.stem import WordNetLemmatizer 
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# To be used with hierarchical clustering
from joblib import Memory

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ranitrajganguly/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ranitrajganguly/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/ranitrajganguly/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Data preprocessing functions

In [2]:
DATASET_PATH = 'test_cases.xlsx'

TYPE = "Type"
KEY = "Key"
CASE_NAME = "Case_Name"
STEPS = "Steps"
STEP_ID = "Step_ID"
LIST_COLUMN = ["Type", "Key", "Case_Name", "Step_ID", "Steps"]

VECTOR_DIMENSION = 200
WORKER_COUNT = 4
WORD_FREQUENCY_THRESHOLD = 2
CONTEXT_WINDOW_SIZE = 2
DOWN_SAMPLING = 0.001

K_MEANS_CLUSTER_COUNT = 25

In [3]:
def get_unique_word_count(data, column_name):
    """
    Calculates the unique wordcount in the dataset from a particular column

    :param data: input data
    :param column_name: name of the column from which unique word count is calculated
    :return: unique word count
    """
    list_words = list()
    column = list(data[column_name])
    for cur_step in column:
        for cur_word in cur_step:
            list_words.append(cur_word)
    return len(set(list_words))

In [4]:
def get_word_frequency(data, column_name):
    """
    Returns the list of words that have a frequency less than two in the dataset

    :param data: input data
    :param column_name: name of the column from which word frequency is calculated
    :return: list of words
    """
    list_words = list()
    column = list(data[column_name])
    for cur_step in column:
        for cur_word in cur_step:
            list_words.append(cur_word)
    unique_words_list = set(list_words)

    dict_word_frequency = {}
    for cur_word in unique_words_list:
        # Initialize count of all words to 0
        dict_word_frequency[cur_word] = 0

    for cur_step in column:
        # Compute frequency of each word
        for cur_word in cur_step:
            dict_word_frequency[cur_word] += 1

    final_list = list()
    # List of words that occur only once
    for word, frequency in dict_word_frequency.items():
        if frequency < 2:
            final_list.append(word)
    return final_list

In [5]:
def read_input_data(data):
    """
    Method to load input data and iterate through it

    :param data: training data
    """
    print("Reading input data")
    cur_index = 0
    for index, row in data.iterrows():
        current_type = row[TYPE]
        current_key = row[KEY]
        current_name = row[CASE_NAME]
        current_step_id = row[STEP_ID]
        current_steps = row[STEPS]

        data.loc[cur_index] = [current_type, current_key, current_name, current_step_id, current_steps]
        cur_index += 1
    print("Shape of data = ", data.shape)

In [6]:
def clean_dataset(data):
    """
    Cleans the input dataset for the test-steps and test-case columns

    :param data: input data
    :return: cleaned dataset
    """
    print("Size of dataset before preprocessing = ", data.shape)

    # Replace URL, paths, HTML tags and convert to lower-case
    data[STEPS] = data[STEPS].apply(lambda x: re.sub(r'http\S+', 'URL', x))
    data[CASE_NAME] = data[CASE_NAME].apply(lambda x: re.sub(r'http\S+', 'URL', x))
    data[STEPS] = data[STEPS].apply(lambda x: re.sub(r'/[\w-]*', '', x))
    data[CASE_NAME] = data[CASE_NAME].apply(lambda x: re.sub(r'/[\w-]*', '', x))
    data[STEPS] = data[STEPS].apply(lambda x: re.sub(r'\{[^)]*}', '', x))
    data[CASE_NAME] = data[CASE_NAME].apply(lambda x: re.sub(r'\{[^)]*}', '', x))
    data[STEPS] = data[STEPS].apply(lambda x: x.lower())
    data[CASE_NAME] = data[CASE_NAME].apply(lambda x: x.lower())

    # Remove digits, punctuations and extra-spaces
    data[STEPS] = data[STEPS].apply(lambda x: re.sub('\w*\d\w*', '', x))
    data[CASE_NAME] = data[CASE_NAME].apply(lambda x: re.sub('\w*\d\w*', '', x))
    data[STEPS] = data[STEPS].apply(
        lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x))
    data[CASE_NAME] = data[CASE_NAME].apply(
        lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x))
    data[STEPS] = data[STEPS].apply(lambda x: re.sub(' +', ' ', x))
    data[CASE_NAME] = data[CASE_NAME].apply(lambda x: re.sub(' +', ' ', x))

    # Tokenization
    data[STEPS] = data[STEPS].apply(lambda x: TweetTokenizer().tokenize(x))
    data[CASE_NAME] = data[CASE_NAME].apply(lambda x: TweetTokenizer().tokenize(x))
    unique_word_count_steps = get_unique_word_count(data, STEPS)
    unique_word_count_case = get_unique_word_count(data, CASE_NAME)
    print(f"Number of unique words across Test-cases = {unique_word_count_case} "
          f"& Test-Steps = {unique_word_count_steps} after Tokenization")

    # Stopword removal
    stop_words = set(stopwords.words('english'))
    data[STEPS] = data[STEPS].apply(lambda x: [w for w in x if not w in stop_words])
    data[CASE_NAME] = data[CASE_NAME].apply(lambda x: [w for w in x if not w in stop_words])
    unique_word_count_steps = get_unique_word_count(data, STEPS)
    unique_word_count_case = get_unique_word_count(data, CASE_NAME)
    print(f"Number of unique words across Test-cases = {unique_word_count_case} "
          f"& Test-Steps = {unique_word_count_steps} after Stopword Removal")

    # Lemmatization for test case-names
    word_net_lemmatizer = WordNetLemmatizer()
    data[STEPS] = data[STEPS].apply(lambda x: [word_net_lemmatizer.lemmatize(w) for w in x])
    data[CASE_NAME] = data[CASE_NAME].apply(lambda x: [word_net_lemmatizer.lemmatize(w) for w in x])

    # Remove words that occur a certain number of times
    word_frequency_threshold_steps = get_word_frequency(data, STEPS)
    word_frequency_threshold_case = get_word_frequency(data, CASE_NAME)
    print(f"Number of words that occurred only once in Test-Cases = {len(word_frequency_threshold_case)}"
          f"& Test-Steps = {len(word_frequency_threshold_steps)}")

    # List of words to be removed in Test-Steps
    for index, row in data.iterrows():
        current_test_name = row[STEPS]
        list_words_to_remove_steps = list()
        for word in current_test_name:
            if word in word_frequency_threshold_case:
                list_words_to_remove_steps.append(word)
        data.loc[index][STEPS] = [elem for elem in current_test_name if
                                            not elem in list_words_to_remove_steps]

    # List of words to be removed in Test-Case
    for index, row in data.iterrows():
        current_test_name = row[CASE_NAME]
        list_words_to_remove_case = list()
        for word in current_test_name:
            if word in word_frequency_threshold_case:
                list_words_to_remove_case.append(word)
        data.loc[index][CASE_NAME] = [elem for elem in current_test_name if
                                                not elem in list_words_to_remove_case]

    # Remove instances with empty names
    data = data.loc[data[STEPS] != '']
    data = data.loc[data[CASE_NAME] != '']

    unique_word_count_steps = get_unique_word_count(data, STEPS)
    unique_word_count_case = get_unique_word_count(data, CASE_NAME)
    print(f"Unique word count in Test-Case = {unique_word_count_case} "
          f"& Unique word count in Test-Steps = {unique_word_count_steps}")

    print("Size of dataset after preprocessing = ", data.shape)
    return data

In [7]:
def return_training_list(data):
    """
    Returns the necessary fields to train the word2vec word embedding model i.e 'type', 'name', 'steps'

    :param data: preprocessed data
    :return: training data
    """
    print("Returning Training list")
    training_list = list()
    for index, row in data.iterrows():
        temp_list = list()

        if not pd.isnull(row[TYPE]):
            temp_list.append(str(row[TYPE]))

        if isinstance(row[CASE_NAME], list):
            for elem in row[CASE_NAME]:
                temp_list.append(elem)
        else:
            if isinstance(row[CASE_NAME], str):
                temp_list.append(row[CASE_NAME])

        if isinstance(row[STEPS], list):
            for elem in row[STEPS]:
                temp_list.append(elem)
        else:
            if isinstance(row[STEPS], str):
                temp_list.append(row[STEPS])

        # List of lists of tokens
        training_list.append(temp_list)
    print(f"Length of list with training data = {len(training_list)}")
    return training_list

# Training Word2Vec Model



In [32]:
def train_word2vec_model(data, pre_trained_model, pre_trained_model_path, vector_size, workers, word_freq_threshold,
                         context_window_size, down_sampling):
    """
    Trains and adds the new corpus into the pretrained word2vec model

    :param data: preprocessed data
    :param pre_trained_model: model
    :param pre_trained_model_path: model path
    :param vector_size: vector size dimension
    :param workers: number of workers
    :param word_freq_threshold: maximum frequency limit for a word
    :param context_window_size: context size of CBOW
    :param down_sampling: down-sampling value
    :return: trained model
    """
    print("Training word2vec model....")
    list_word_vector_median = list()

    # Initialize model
    my_model = Word2Vec(
        vector_size=vector_size,
        workers=workers,
        min_count=word_freq_threshold,
        window=context_window_size,
        sample=down_sampling
    )

    # Build model and count total number of examples
    my_model.build_vocab(data)
    total_examples = my_model.corpus_count
    all_words = list(my_model.wv.index_to_key)
    for cur_word in all_words:
        my_model.wv[cur_word] = np.zeros(VECTOR_DIMENSION)

    # Update vocabulary with our corpus
    my_model.build_vocab([list(pre_trained_model.index_to_key)], update=True)
    my_model.wv.vectors_lockf = np.ones(len(my_model.wv))
    my_model.wv.intersect_word2vec_format(pre_trained_model_path, binary=True, lockf=1.0)

    # Get Mean & Standard-Deviation of initialized word vectors
    for cur_word in all_words:
        if any(my_model.wv[cur_word] != 0):
            list_word_vector_median.append(np.median(my_model.wv[cur_word]))

    # Initialize Mean & Standard-Deviation for normal distributions of medians
    mu = np.mean(list_word_vector_median)
    sigma = np.std(list_word_vector_median)

    # Initialize the remaining word vectors that are not present in the pre-trained word2vec model
    for cur_word in all_words:
        if all(my_model.wv[cur_word] == 0):
            new_word_vector = np.random.normal(mu, sigma, 200)
            my_model.wv[cur_word] = new_word_vector

    # Train the model
    my_model.train(data, total_examples=total_examples, epochs=25)
    return my_model

In [22]:
def return_data_tuple(data):
    """
    Returns tuples with step_id, step which is used to retrieve the step_id after clustering

    :param data: data
    :return: list_tuple_step_id, list_test_step_clustering
    """
    print("Returning data tuple")
    list_tuple_step_id = list()
    list_test_step_clustering = list()

    for index, row in data.iterrows():
        step_id = row[STEP_ID]
        step = row[STEPS]
        list_tuple_step_id.append((step_id, step))

        temp_list = list()
        if isinstance(row[STEPS], list):
            for cur_element in row[STEPS]:
                temp_list.append(cur_element)
        else:
            if isinstance(row[STEPS], str):
                temp_list.append(row[STEPS])
        list_test_step_clustering.append(temp_list)

    print(f"First Tuple = {list_tuple_step_id[0]}")
    return list_tuple_step_id, list_test_step_clustering

# Computing Similarity using Word-Mover's Distance

In [10]:
def initialize_similarity_matrix(list_test_step_clustering):
    """
    Create and returns an empty matrix with rows and columns equal to the shape of 'list_test_step_clustering'

    :param list_test_step_clustering: clustering steps list
    :return: matrix_similarity_distance
    """
    print("Initializing Similarity Matrix")
    rows = columns = len(list_test_step_clustering)
    return np.zeros((rows, columns))

In [11]:
def compute_and_save_similarity_distance(model, list_test_step_clustering, matrix_similarity_distance):
    """
    Computes the similarity distance between the rows and columns of list_test_step_clustering
    and saves the result in a .txt file

    :param model: word2vec model
    :param list_test_step_clustering: clustering steps list
    :param matrix_similarity_distance: empty matrix
    :return: matrix_similarity_distance with similarities
    """
    print("Computing Similarity using WMD and filling Similarity Matrix")
    rows = columns = len(list_test_step_clustering)

    for cur_row in range(rows):
        for cur_column in range(cur_row, columns):
            similarity_distance = model.wv.wmdistance(list_test_step_clustering[cur_row],
                                                      list_test_step_clustering[cur_column])
            # Upper limit
            if similarity_distance > 800:
                similarity_distance = 800
            matrix_similarity_distance[cur_row, cur_column] = matrix_similarity_distance[
                cur_column, cur_row] = similarity_distance

    # Save the similarity matrix
    path_save_matrix_similarity = 'matrix_similarity.txt'
    np.savetxt(path_save_matrix_similarity, matrix_similarity_distance)

    return matrix_similarity_distance

In [26]:
def compute_average_test_step(test_step, model, pre_trained_model):
    """
    Computes the average word vector of the test step

    :param test_step: current test-step
    :param model: trained word2vec model
    :param pre_trained_model: pre-trained word2vec model
    :return: average
    """
    list_word_vectors = list()
    count_word = 0
    for word in test_step:
        try:
            list_word_vectors.append(model[word])
        except:
            try:
                list_word_vectors.append(pre_trained_model[word])
            except:
                return np.zeros(200)
        count_word += 1
    sum_vectors = sum(list_word_vectors)
    average = sum_vectors / count_word
    return average

# Clustering using K-Means

In [27]:
def perform_clustering_and_save(model, pre_trained_model, data, list_tuple_step_id, list_test_step_clustering):
    """
    Performs k-means clustering on the test_step and step_id and saves the clustered results in .txt files

    :param model: trained word2vec model
    :param pre_trained_model: pre-trained word2vec model
    :param data: processed data
    :param list_tuple_step_id: tuple of step-id
    :param list_test_step_clustering: clustered list
    :return: dict_clusters: dictionary of the clusters
    """
    labels_list_kmeans = list()

    # Compute average word vector to be used in k-means
    avg_word_sentence_vectors = list()
    for cur_test_step in list_test_step_clustering:
        if len(cur_test_step) > 0:
            avg_word_sentence_vectors.append(
                compute_average_test_step(
                    cur_test_step,
                    model,
                    pre_trained_model
                )
            )

    # Initialize K-means
    k_means = KMeans(
        n_clusters=K_MEANS_CLUSTER_COUNT,
        init='k-means++',
        max_iter=500
    )
    k_means.fit(avg_word_sentence_vectors)
    labels = k_means.labels_
    labels_list_kmeans.append(labels)

    dict_clusters = {}
    for cur_label in set(labels):
        label_indices = np.where(labels == cur_label)[0].tolist()
        for cur_index in label_indices:
            dict_clusters[int(list_tuple_step_id[cur_index][0])] = cur_label

    print(f"Number of clusters = {k_means.n_clusters} & Number of labels = {k_means.labels_}")
    save_clusters(k_means.labels_, list_tuple_step_id, list_test_step_clustering, data)
    save_cluster_labels(k_means.labels_, list_tuple_step_id)

    return dict_clusters

In [30]:
def save_clusters(labels, list_tuple_step_id, list_test_step_clustering, data):
    """
    Saves clusters in a text-file

    :param labels: k-means labels
    :param list_tuple_step_id: tuple of step-id
    :param list_test_step_clustering: clustered list
    :param data: processed data
    """
    path = "clustered_data_kmeans.txt"
    output = open(path, "a")
    for label in set(labels):
        indices_label = np.where(labels == label)[0].tolist()
        for index in indices_label:
            str_to_save = "[" + str(label) + "]:\t\t" + data.loc[index]["Key"] + "\t\t" + str(
                list_tuple_step_id[index][0]) + "\t\t" + str(list_test_step_clustering[index]) + "\n"
            output.write(str_to_save)
    print("Successfully generated text file with title = clustered_data_kmeans.txt")
    output.close()

In [31]:
def save_cluster_labels(labels, list_tuple_step_id):
    """
    Saves cluster labels in a text-file

    :param labels: k-means labels
    :param list_tuple_step_id: tuple of step-id
    """
    path = "clustered_labels_kmeans.txt"
    output = open(path, "a")
    for single_label in set(labels):
        indices_label = np.where(labels == single_label)[0].tolist()
        str_to_save = "[" + str(single_label) + "]: " + ','.join(
            str(list_tuple_step_id[x][0]) for x in indices_label) + "\n"
        output.write(str_to_save)
    print("Successfully generated text file with title = clustered_labels_kmeans.txt")
    output.close()

# Code Execution

In [35]:
# Load dataset
print("----START----")

training_data = pd.read_excel(DATASET_PATH)

# Preprocess the training dataset
read_input_data(training_data)
test_steps_df = clean_dataset(training_data)
training_list = return_training_list(test_steps_df)

# Loading pre-trained on 15GB of Stack Overflow posts Word2Vec Model
pre_trained_model_path = "SO_vectors_200.bin"
pre_trained_model = KeyedVectors.load_word2vec_format(
    pre_trained_model_path,
    binary=True
)

# Train the word2vec model using the new dataset
my_model = train_word2vec_model(
    training_list,
    pre_trained_model,
    pre_trained_model_path,
    VECTOR_DIMENSION,
    WORKER_COUNT,
    WORD_FREQUENCY_THRESHOLD,
    CONTEXT_WINDOW_SIZE,
    DOWN_SAMPLING
)

# Obtain tuple for step_id and list of test-step clustering from the training data
list_tuple_step_id, list_test_step_clustering = return_data_tuple(test_steps_df)

# Create an Empty matrix with rows and columns equal to the shape of 'list_test_step_clustering'
matrix_similarity_distance = initialize_similarity_matrix(list_test_step_clustering)

# Calculate similarity between word embeddings using Word-Movers Distance and save the result
matrix_similarity_distance = compute_and_save_similarity_distance(
    my_model,
    list_test_step_clustering,
    matrix_similarity_distance
)

# Perform Clustering of test-steps using K-Means
dict_clusters = perform_clustering_and_save(
    my_model,
    pre_trained_model,
    test_steps_df,
    list_tuple_step_id,
    list_test_step_clustering
)

print("----DONE----")

----START----
Reading input data
Shape of data =  (369, 5)
Size of dataset before preprocessing =  (369, 5)
Number of unique words across Test-cases = 352 & Test-Steps = 818 after Tokenization
Number of unique words across Test-cases = 310 & Test-Steps = 742 after Stopword Removal
Number of words that occurred only once in Test-Cases = 16& Test-Steps = 331
Unique word count in Test-Case = 281 & Unique word count in Test-Steps = 683
Size of dataset after preprocessing =  (369, 5)
Returning Training list
Length of list with training data = 369
Training word2vec model....
Returning data tuple
First Tuple = (1.0, ['enter', 'phone', 'number'])
Initializing Similarity Matrix
Computing Similarity using WMD and filling Similarity Matrix
Number of clusters = 25 & Number of labels = [ 5 11 17  5 17  5 11  1  5 11 12 12  5 18  5 11 11 18  5 11 19 18  5 17
 15 15  1  7  1  7 20  7  1  7  1  7 20  7 20  7  2  7 16 16 16  3  2  7
 16 16 16  3  2  7 16 16 16  3 13 10 14 19 14 14 14 14 14 14  6 14 13 